# Wikipedia Notable Life Expectancies

# [Notebook 2 of 4: Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean_thanak_2022_06_13.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the wp_life_expect_raw_complete dataset
conn = sql.connect("wp_life_expect_raw_complete.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_raw_complete", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133900 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133898,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133899,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
86808,December 2016,19,William H. Hudnut III,", 84, American politician, Member of the United States House of Representatives from Indiana's 11th congressional district (1973–1975), Mayor of I...",https://en.wikipedia.org/wiki/William_H._Hudnut_III,34
37994,August 2008,15,James Orthwein,", 84, American businessman, cancer.",https://en.wikipedia.org/wiki/James_Orthwein,6
35186,November 2007,21,Tom Johnson,", 79, Canadian Hall of Fame hockey player, heart failure.",https://en.wikipedia.org/wiki/Tom_Johnson_(ice_hockey),2
99427,September 2018,28,Joe Masteroff,", 98, American playwright (Tony, 1967), winner ().",https://en.wikipedia.org/wiki/Joe_Masteroff,14
130498,January 2022,22,Katuutire Kaura,", 80, Namibian politician, MP (1990–2015).",https://en.wikipedia.org/wiki/Katuutire_Kaura,12


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,900 rows and 6 columns.
- `month_year` contains the month and year of death, while `day` contains the day of the month of death.
- `name` is the notable person's name.  It is a nominal feature that will not be used for analysis, but will be maintained for any referencing needs.
- `info` contains multiple items including the notable person's "age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, (and) cause of death (if known)."
- `link` is the url to the notable person's individual Wikipedia page.  If such a page does not exist, there is either a non-working link (https://en.wikipedia.orgNone), or the link is to a page with a message that the page does not exist for that individual.  `link` is a unique identifier for all entries, except the 6 with the non-working link, which do have unique `name` values from each other.
- `num_references` contains the number of references on the notable person's individual Wikipedia page.  This feature serves as a proxy measure of notability.
- Prior to EDA, our task will be to extract the individual elements that are comined in `month_year` and `info` columns.

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

In [6]:
# Checking duplicate rows
df.duplicated().sum()

0

<IPython.core.display.Javascript object>

In [7]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year       0.000
day              0.000
name             0.004
info             0.000
link             0.000
num_references   0.000
dtype: float64

<IPython.core.display.Javascript object>

In [8]:
# Checking number of missing values per row (not necessary here, but done to keep process standard)
df.isnull().sum(axis=1).value_counts()

0    133894
1         6
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- Our dataset was saved to and read from the database without any hiccups.
- As expected, we have 6 entries that are missing `name`, but we will find it in their `info` values.
- All columns are currently of object type.  We will need to appropriately typecast them after separating the information in `month_year` and `info`.

## Data Cleaning

### Addressing Missing `name` Values

In [9]:
# Checking rows with missing name values
missing_name = df[df["name"].isna()]
missing_name

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- These rows vary from the main set as there is a substring containing the person's name at the start of the `info` string.
- As there are so few rows missing `name`, let us address this issue first.

In [10]:
# For loop to copy name value from info column
treat_rows = missing_name.index
for i in treat_rows:
    info = df.loc[i, "info"]
    info_lst = info.split(sep=",", maxsplit=1)
    df.loc[i, "name"] = info_lst[0]

# Re-check rows
df.loc[treat_rows, :]

,month_year,day,name,info,link,num_references
18937,August 2001,11,Kevin Kowalcyk,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,Vincent Palmer,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,Barry Stigler,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,Nana Gualdi,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,Scott Workman,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,Thami Shobede,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- Missing `name` values have been addressed.
- The names still appear in the `info` value for these rows, but we can address that as we separate the information in that column.

In [11]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133900 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- We have no remaining missing values.
- Let us treat the `month_year` column next.

### Separating `month` and `year`

In [12]:
# Separating month and year into 2 columns and typecasting year as integer
df.loc[:, "year"] = df["month_year"].apply(lambda x: x.split(sep=" ")[1])
df["year"] = df["year"].astype("int64")

df.loc[:, "month"] = df["month_year"].apply(lambda x: x.split(sep=" ")[0])
df.head(2)

,month_year,day,name,info,link,num_references,year,month
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January


<IPython.core.display.Javascript object>

In [13]:
# Dropping month_year column
df.drop("month_year", axis=1, inplace=True)

<IPython.core.display.Javascript object>

### Treating `info`

#### Checking a Sample

In [52]:
# Checking a sample of info
df.sample(5)

,day,name,info,link,num_references,year,month
73068,3,Hubert Egger,"87, German cross country skier",https://en.wikipedia.org/wiki/Hubert_Egger,0,2014,December
62151,3,Cedric Brooks,"70, Jamaican musician (The Skatalites), cardiac arrest",https://en.wikipedia.org/wiki/Cedric_Brooks,5,2013,May
83550,16,Candy Ruff,"65, American politician, member of the Kansas House of Representatives (1993–2009)",https://en.wikipedia.org/wiki/Candy_Ruff,1,2016,June
59922,12,Jean Krier,"64, Luxembourgian poet",https://en.wikipedia.org/wiki/Jean_Krier,4,2013,January
81319,21,Ivan M. Matheson,"89, American politician",https://en.wikipedia.org/wiki/Ivan_M._Matheson,5,2016,February


<IPython.core.display.Javascript object>

#### Observations:
- We have leading and trailing characters of comma, period, and white space that can be removed.

#### Removing Leading and Trailing Commas, Whitespace, and Periods

In [53]:
# Removing the leading/trailing commas, periods, and whitespace
df.loc[:, "info"] = df["info"].apply(lambda x: x.strip(" ,."))
df.sample(5)

,day,name,info,link,num_references,year,month
54635,17,Matt Conte,"85, American football player and coach",https://en.wikipedia.org/wiki/Matt_Conte,2,2012,March
68707,20,Yoshio Shinozuka,"90, Japanese Imperial Army soldier (Unit 731)",https://en.wikipedia.org/wiki/Yoshio_Shinozuka,2,2014,April
116452,1,Derek Mahon,"78, Irish poet",https://en.wikipedia.org/wiki/Derek_Mahon,12,2020,October
35479,23,Ruth Wallis,"87, American singer-songwriter, complications of Alzheimer's disease",https://en.wikipedia.org/wiki/Ruth_Wallis,0,2007,December
45019,5,Lothar Engelhardt,"70, German military scientist",https://en.wikipedia.org/wiki/Lothar_Engelhardt,1,2010,April


<IPython.core.display.Javascript object>

#### Checking the Set of First Substrings (Before First Comma)

In [80]:
# Checking values in the first substring of info (before the first comma)
age_set = set(df["info"].apply(lambda x: x.split(sep=",", maxsplit=1)[0]))
age_set

{'3-months',
 'Mexican journalist',
 '11',
 'British peer and military officer',
 'Tunisian politician and judge',
 'Iranian Islamic scholar',
 'Sri Lanka Army officer',
 'Norwegian architect',
 '48',
 'Syrian politician and literary personality',
 'American poet and Mormon hymnwriter',
 'Sarah-Jayne Mulvihill',
 'Cambodian-born Theravada monk and teacher',
 'Guinea-Bissauan politician',
 'Indian novelist',
 'Israeli Haredi rabbi and orator',
 'Provisional Irish Republican Army volunteer',
 'Spanish writer',
 'Ethiopian Chief of Security and Immigration',
 '32',
 'Japanese composer (',
 'Emirati royal',
 '89–90',
 'British-born Italian musician and musicologist',
 'Togolese lawyer and guitarist (Orchestra Baobab)',
 'Abkhaz politician',
 'Canadian journalist',
 '85/86',
 '81. English sports journalist',
 'Brazilian draughts player',
 'Canadian gangster and drug trader',
 '100',
 'French doctor and drug test pioneer',
 '(岡本喜八)',
 '81-82',
 'Nigerian film producer and director',
 'Ivoria

<IPython.core.display.Javascript object>

#### Observations:
- Though we can see the age values, there is a lot of other information in the first substring of the `info` value.
- Some entries may be missing the age information.
- Also, age is entered in various formats.
- Let us see how many variations there are.

In [81]:
# Checking number of different values for first substring of info column
len(age_set)

1412

<IPython.core.display.Javascript object>

#### Observations:
- There are 1412 variations for the substring that represents age on Wikipedia.  
- We are likely better off with a regular expressions approach to dividing this string feature, rather than using Python string methods alone.